In [1]:
import numpy as np
import pandas as pd

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [2]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

--2024-10-27 16:17:41--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.107.207, 74.125.197.207, 74.125.195.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.107.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: 'spa-eng.zip'

spa-eng.zip         100%[===================>]   2.52M  --.-KB/s    in 0.008s  

2024-10-27 16:17:41 (326 MB/s) - 'spa-eng.zip' saved [2638744/2638744]



In [3]:
text_file = "spa-eng/spa.txt"

with open(text_file) as f:
    lines = f.read().split("\n")[:-1]

text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start]" + spanish + "[end]"
    text_pairs.append((english,spanish))

In [4]:
text_pairs[0]

('Go.', '[start]Ve.[end]')

In [5]:
import random
print(random.choice(text_pairs))

('What did you go there for?', '[start]¿Para qué fuiste allá?[end]')


In [6]:
random.shuffle(text_pairs)

num_val_samples = int(0.15*len(text_pairs))
num_train_samples = len(text_pairs) -2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [7]:
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[","")
strip_chars = strip_chars.replace("]","")

strip_chars

'!"#$%&\'()*+,-./:;<=>?@\\^_`{|}~¿'

In [8]:
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
max_tokens= vocab_size,
output_mode = "int",
output_sequence_length = sequence_length
)

target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

In [9]:
train_pairs[:1]

[('Get into the car and wait for me.',
  '[start]Metete en el auto y esperame.[end]')]

In [10]:
t = [("hello ","hi "),("hello ","hi ")]

for x,y in zip(*train_pairs[:2]):
    print(x,y)

Get into the car and wait for me. They're talking.
[start]Metete en el auto y esperame.[end] [start]Están hablando.[end]


In [11]:
batch_size = 64

def format_dataset(eng,spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    
    return({
        "english": eng,
        "spanish": spa[:,:-1],
    },spa[:,1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts,spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

In [12]:
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [13]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


In [14]:
for inputs, targets in train_ds.take(1):
    print(inputs,targets)

{'english': <tf.Tensor: shape=(64, 20), dtype=int64, numpy=
array([[136, 165, 808, ...,   0,   0,   0],
       [ 13,   8, 400, ...,   0,   0,   0],
       [ 34, 138, 616, ...,   0,   0,   0],
       ...,
       [  3,  72,  36, ...,   0,   0,   0],
       [112,  40,  63, ...,   0,   0,   0],
       [ 15,   5,  18, ...,   0,   0,   0]])>, 'spanish': <tf.Tensor: shape=(64, 20), dtype=int64, numpy=
array([[1881,    1,   98, ...,    0,    0,    0],
       [  44,   14,  686, ...,    0,    0,    0],
       [  52,  713, 1005, ...,    0,    0,    0],
       ...,
       [ 507,   14, 6714, ...,    0,    0,    0],
       [4798,   84, 6977, ...,    0,    0,    0],
       [ 338,  187,   11, ...,    0,    0,    0]])>} tf.Tensor(
[[   1   98 6028 ...    0    0    0]
 [  14  686   59 ...    0    0    0]
 [ 713 1005    2 ...    0    0    0]
 ...
 [  14 6714    2 ...    0    0    0]
 [  84 6977    2 ...    0    0    0]
 [ 187   11  393 ...    0    0    0]], shape=(64, 20), dtype=int64)


In [15]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,),dtype="int64",name="english")
x = layers.Embedding(vocab_size,embed_dim,mask_zero=True)(source)
encoded_source = layers.Bidirectional(layers.GRU(latent_dim),merge_mode="sum")(x)

In [16]:
past_target = keras.Input(shape=(None,),dtype="int64",name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

In [17]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 153s 113ms/step - accuracy: 0.7475 - loss: 6.0312 - val_accuracy: 0.7793 - val_loss: 4.5706
Epoch 2/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 146s 113ms/step - accuracy: 0.6416 - loss: 4.5964 - val_accuracy: 0.7145 - val_loss: 3.8928
Epoch 3/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 147s 113ms/step - accuracy: 0.5062 - loss: 3.9896 - val_accuracy: 0.6379 - val_loss: 3.4639
Epoch 4/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 147s 113ms/step - accuracy: 0.4740 - loss: 3.5475 - val_accuracy: 0.6064 - val_loss: 3.1743
Epoch 5/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 147s 113ms/step - accuracy: 0.5059 - loss: 3.2034 - val_accuracy: 0.5378 - val_loss: 2.9631
Epoch 6/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 147s 113ms/step - accuracy: 0.4889 - loss: 2.9168 - val_accuracy: 0.4973 - val_loss: 2.8067
Epoch 7/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 147s 113ms/step - accuracy: 0.4763 - loss: 2.6845 - val_accuracy: 0.5137 - val_loss: 2.6872
Epoch 8/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 147s 113ms/step - ac

In [18]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
It's said that the Vikings discovered America before Columbus did.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[start] que la [UNK] se [UNK] antes de las [UNK]  que la lo había mal[end]    de de
-
We're sick.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/s